In [ ]:
import tensorflow as tf
import numpy as np

# Memuat model TFLite
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Mendapatkan input dan output tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Menyiapkan input tensor (contoh input acak untuk uji coba)
input_shape = input_details[0]['shape']
input_data = np.random.random_sample(input_shape).astype(np.float32)

# Menjalankan model
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Mendapatkan hasil output
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Output:", output_data)


Output: [[0.03681043 0.0427701  0.03668127 0.04239451 0.03623249 0.03647828
  0.04192922 0.03692108 0.04239915 0.0349411  0.03910677 0.03881742
  0.03374734 0.0358602  0.0372227  0.04232617 0.03681127 0.03148725
  0.03749875 0.03930042 0.04253514 0.04105419 0.04146668 0.0368722
  0.03806244 0.04027347]]


In [ ]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model('word_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Menggunakan operator bawaan TFLite
    tf.lite.OpsSet.SELECT_TF_OPS    # Opsional jika ada ops TensorFlow lainnya
]
converter.experimental_new_converter = True
converter.target_spec.experimental_select_user_tf_ops = False
tflite_model = converter.convert()

with open("model_word.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp3c86zp9_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 26), dtype=tf.float32, name=None)
Captures:
  137828343662192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343737408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343739168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343734768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343737936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343742160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343736000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343739520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343739344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343741280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137828343740

In [ ]:
import tensorflow as tf

# Load TFLite model and extract metadata
interpreter = tf.lite.Interpreter(model_path="model_word.tflite")
interpreter.allocate_tensors()

# Check input details
print(interpreter.get_tensor_details())


[{'name': 'serving_default_input_layer_1:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'arith.constant', 'index': 1, 'shape': array([1280], dtype=int32), 'shape_signature': array([1280], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'arith.constant1', 'index': 2, 'shape': array([960], dtype=int32), 'shape_signature': array([960], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], d

Model tidak menyimpan nama kelas secara langsung.


In [ ]:
!pip install tflite-support-nightly

In [ ]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

In [ ]:
ImageClassifierWriter = image_classifier.MetadataWriter
_MODEL_PATH = "model_word.tflite"
# Task Library expects label files that are in the same format as the one below.
_LABEL_FILE = "word.txt"
_SAVE_TO_PATH = "model_word_metadata.tflite"
# Normalization parameters are required when reprocessing the image. It is
# optional if the image pixel values are in the range of [0, 255] and the input
# tensor is quantized to uint8. See the introduction for normalization and
# quantization parameters below for more details.
# https://www.tensorflow.org/lite/models/convert/metadata#normalization_and_quantization_parameters)
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],
    [_LABEL_FILE])

# Verify the metadata generated by the metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

{
  "name": "ImageClassifier",
  "description": "Identify the most prominent object in the image from a known set of categories.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
      "output_tensor_metadata": [
        {
          "name": "proba